In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ..

/home/kris/git/vq_vae_text


In [3]:
import torch

from pathlib import Path
from vq_vae_text.datasets import ByteLevelTextDataset

In [4]:
dataset = ByteLevelTextDataset('/home/kris/data/text/sent-grams/splits/uniform/bigrams/large/train.txt', seq_len=256)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=16)

Using cached file .cache/637093ea5b36dc6cc1652d157897f500e548d98437e30c81e84823aa4ef18265.p


In [6]:
model_path = Path('results/2020-04-17_18-12-01-textcnn_vq2/checkpoints/model_0.pth')

model = torch.load(model_path).cpu()
model.eval();

In [7]:
batch = next(iter(dataloader))

In [8]:
batch

tensor([[ 68,  97,  32,  ...,   0,   0,   0],
        [ 65, 117, 115,  ...,   0,   0,   0],
        [ 83, 105, 101,  ...,   0,   0,   0],
        ...,
        [ 65, 108, 115,  ...,   0,   0,   0],
        [ 68, 101, 110,  ...,   0,   0,   0],
        [ 68, 105, 101,  ...,   0,   0,   0]])

In [9]:
z, _, codes = model.encode(batch)

In [14]:
print(decode(codes))

['Da ein ich aus Verbehen itritten einer Landruppe mit Wem Jag am Stadion angekommen. Die Einheimischen haben unt mit weichnhageln beschossen und mit Steiten die Scheiden im Zugieingeschmussen.', 'Ausschliesslich an Entwickter hat SPm einen al. Fei aussa bezeichneten Pranitak ausverieiert. Irite Fests mit weiteren Getaten, die Landan und Fe als heisten sollen, laufen angeblich bereits.', 'Sie liege auf der Mille dessen, was die Maschafe immer gesagt hatten. Es gebe ein eigenstandiges Arbeitsrecht fur die Keicher und die Mitarbeiter wirsten, dass sie sich darit zu halten haben.', 'Die Zahren seien zwar im Jahren der Erwartungen aufgefallen. Allerdings gebe es jeder bei den Ergebnissen nach beim ausblich irgergleuche aussergewohnlichen Uberraschunden, sagte ein Mindler.', 'Die Zahren seien zwar im Jahren der Erwartungen aufgefallen. Allerdings gebe es jeder bei den Ergebnissen nach beim ausblich irgergleuche aussergewohnlichen Uberraschunden, sagte ein Mindler.', 'Hambung verdient besser

In [11]:
def decode_code(codes):
    code_a, code_b = codes
    
    quant_a = model.quantize_a.quantize.embed_code(code_a)
    quant_a = quant_a.view(quant_a.size(0), quant_a.size(1) // model.quantize_a.d_slice, -1)
    quant_a = quant_a.permute(0, 2, 1)

    quant_b = model.quantize_b.quantize.embed_code(code_b)
    quant_b = quant_b.view(quant_b.size(0), quant_b.size(1) // model.quantize_b.d_slice, -1)
    quant_b = quant_b.permute(0, 2, 1)

    dec = model.decode((quant_a, quant_b))

    return dec

In [123]:
def decode_code(codes):
    code_a, code_b = codes
    
    quant_a = model.quantize_a.embed_code(code_a)
    quant_a = quant_a.permute(0, 2, 1)

    quant_b = model.quantize_b.embed_code(code_b)
    quant_b = quant_b.permute(0, 2, 1)

    dec = model.decode((quant_a, quant_b))

    return dec

In [13]:
def decode(codes):
    dec = decode_code(codes).argmax(-1)
    return dataset.seq_to_text(dec)

In [116]:
def decode_interpol(z_a, z_b, a):
    codes = []
    for a, b in zip(z_a, z_b):
        z = a * (1 - a) + b * a
        model.encode

In [18]:
z, _, codes = model.encode(batch)

for i in range(64):
    codes[1][:, 1] = i
    #codes[1][:, 10] = i % 10
    
    if i % 10 == 0:
        print()
    
    print(f'{i}.\t', decode(codes)[0])


0.	 Da ein ich aus Vanzigen armieten einer Landruppe mit Wem Jag am Stadion angekommen. Die Einheimischen haben unt mit weichnhageln beschossen und mit Steiten die Scheiden im Zugieingeschmussen.
1.	 Da ein ich aus Varzenen armenten einer Landruppe mit Wem Jag am Stadion angekommen. Die Einheimischen haben unt mit weichnhageln beschossen und mit Steiten die Scheiden im Zugieingeschmussen.
2.	 Da ein ich aus hinzegen ernieten einer Landruppe mit Wem Jag am Stadion angekommen. Die Einheimischen haben unt mit weichnhageln beschossen und mit Steiten die Scheiden im Zugieingeschmussen.
3.	 Da ein ich aus Vorbehen ernieten einer Landruppe mit Wem Jag am Stadion angekommen. Die Einheimischen haben unt mit weichnhageln beschossen und mit Steiten die Scheiden im Zugieingeschmussen.
4.	 Da ein ich aus Vorbenen urmetten einer Landruppe mit Wem Jag am Stadion angekommen. Die Einheimischen haben unt mit weichnhageln beschossen und mit Steiten die Scheiden im Zugieingeschmussen.
5.	 Da ein ich aus 